In [1]:
import os
import json
from PIL import Image

# Extracting different image file names

In [2]:
# Location of the directory containing the dataset images and the json files
datasetDir = "../../assets/Takshak_dataset40%" 
filenames = os.listdir(datasetDir)
imagenames = []
for file in filenames:
    if file.split('.')[-1] == 'png':
        imagenames.append(file)

In [3]:
# Directories where the processed images and json files will be stored
imgSaveDir = "../../assets/Preprocessed_Takshak_dataset/img"
jsonSaveDir = "../../assets/Preprocessed_Takshak_dataset/json"
if not os.path.exists(imgSaveDir): os.makedirs(imgSaveDir)
if not os.path.exists(jsonSaveDir): os.makedirs(jsonSaveDir)

In [4]:
for name in imagenames:
    img = Image.open(os.path.join(datasetDir, name)) # read the image
    width, height = img.size

    jsonName = name.split('.')[0] + ".json"
    f = open(os.path.join(datasetDir, jsonName)) # opened the json file
    coordinates = json.load(f) # read the data
    f.close() # closed the file

    # Training dataset images are 1500 x 1500 in size

    # Manipulate width
    newWid = 1500 
    if width <= 1500 or (coordinates['Start'][0] < 1500 and coordinates['End'][0] < 1500):
        left = 0
    elif (coordinates['Start'][0] - coordinates['End'][0]) < 1500:
        left = (coordinates['Start'][0] + coordinates['End'][0]) // 2 - 749
    # else if (coordinates['Start'][0] - coordinates['End'][0]) >= 1500: we are doomed

    # Manipulate height
    newHig = 1500
    if height <= 1500 or (coordinates['Start'][1] < 1500 and coordinates['End'][1] < 1500): top = 0
    elif (coordinates['Start'][1] - coordinates['End'][1]) < 1500:
        top = (coordinates['Start'][1] + coordinates['End'][1]) // 2 - 749
    # else if (coordinates['Start'][1] - coordinates['End'][1]) >= 1500: we are doomed

    # Crop/extend the image
    img1 = img.crop((left, top, left + newWid, top + newHig))

    # convert transparent pixels to white
    pixData = img1.getdata()
    newData = []
    for item in pixData:
        if item[3] == 0: item = (255, 255, 255, 255)
        newData.append(item)
    img1.putdata(newData)
    img1.convert("RGB")

    img1.save(os.path.join(imgSaveDir, name)) # save the image

    # Creating a json file and writing coordinate shifts in it 
    jsonData = {}
    jsonData["Shift"] = (left, top)
    jsonObj = json.dumps(jsonData, indent = 4)
    f = open(os.path.join(jsonSaveDir, jsonName), 'w')
    f.write(jsonObj)
    f.close() 
